# Examining all files in RecordSearch with the access status of 'Closed'
(Harvested on 1 January 2021)

This notebook attempts some large-scale analysis of files from the National Archives of Australia's RecordSearch database that have the access status of 'closed'. For a previous attempt at this, see [Closed Access](http://closedaccess.herokuapp.com/). For more background, see my [*Inside Story* article](https://insidestory.org.au/withheld-pending-advice/) from 2018.

See [this notebook](harvest_closed_files.ipynb) for the code used to harvest the data and create the CSV dataset.

In [1]:
import datetime

import altair as alt
import pandas as pd

The harvested data has been saved as a CSV file. First we'll open it up using Pandas.

In [2]:
df2020 = pd.read_csv(
    "data/closed-20210101.csv",
    parse_dates=["contents_start_date", "contents_end_date", "access_decision_date"],
    keep_default_na=False,
)
df2020.head()

,identifier,series,control_symbol,title,contents_date_str,contents_start_date,contents_end_date,location,access_status,access_decision_date_str,access_decision_date,reasons
0,65454,A373,5117,[Home Office suspect index],1946 - 1950,1946-01-01,1950-01-01,Canberra,Closed,26 Nov 1980,1980-11-26,33(1)(b)
1,66910,A432,1929/394 ATTACHMENT 2,Abrahams. Opinions.,1927 - 1928,1927-01-01,1928-01-01,Canberra,Closed,30 Jul 2018,2018-07-30,33(3)(a)(i)|33(3)(b)
2,66911,A432,1929/394 ATTACHMENT 3,"Legal opinions expressed by O Dixon, E Gorman ...",1928 - 1928,1928-01-01,1928-01-01,Canberra,Closed,30 Jul 2018,2018-07-30,33(3)(a)(i)|33(3)(b)
3,99746,A471,49941,[THOMAS Leslie Hector (Leading Aircraftman) : ...,1943 - 1943,1943-01-01,1943-01-01,Canberra,Closed,20 May 1999,1999-05-20,33(1)(g)
4,103094,A518,FJ118/6,Nauru Census 1952,1952 - 1953,1952-01-01,1953-01-01,Canberra,Closed,24 Oct 1989,1989-10-24,33(1)(d)|33(1)(g)


How many closed files are there?

In [3]:
df2020.shape[0]

11140

## What series do the 'closed' files come from?

First let's see how many different series are represented in the data set.

In [4]:
df2020["series"].unique().shape[0]

686

Now let's look at the 25 most common series.

In [5]:
df2020["series"].value_counts()[:25]

K60        1671
A1838      1571
A13147      581
A6122       322
AWM54       309
A1209       293
A9737       232
B26         196
A1533       173
D4082       162
B73         158
A6135       154
E72         151
F1          126
AWM239      118
A432        114
PP946/1     112
A7452       103
A7324        85
C4384        76
C139         76
A3092        73
A2539        72
A1200        72
D1915        70
Name: series, dtype: int64

Series [A1838](http://recordsearch.naa.gov.au/scripts/AutoSearch.asp?Number=A1838) is familiar to anyone who's looked into the NAA's access examination process. It's a general correspondence series from DFAT, and requests for access tend to take a **long** time to be processed. Series [K60](http://recordsearch.naa.gov.au/scripts/AutoSearch.asp?Number=K60) contains repatriation files from the Department of Veterans' Affairs, so these will often been withheld on privacy grounds. We'll see more about both of these below.

Let's chart the results.

In [6]:
# This creates a compact dataset to feed to Altair for charting
# We could make Altair do all the work, but that would embed a lot of data in the notebook.

# Save the series counts to a new dataframe.
series_counts = df2020["series"].value_counts().to_frame().reset_index()
series_counts.columns = ["series", "count"]

In [7]:
# Chart the results, sorted by number of files
alt.Chart(series_counts[:50]).mark_bar().encode(
    x=alt.X("series", sort="-y"),
    y=alt.Y("count", title="number of files"),
    tooltip=["series", "count"],
)

alt.Chart(...)

This is only the top 50 of 686 series, so quite obviously there's a very long tail of series that have a small number of closed files.

## What reasons are given for closing the files?

[Section 33](http://www8.austlii.edu.au/cgi-bin/viewdoc/au/legis/cth/consol_act/aa198398/s33.html) of the Archives Act defines a number of 'exemptions' – these are reasons why files should not be opened to public access. These reasons are recorded in RecordSearch, so we can explore why files have been closed. It's a little complicated, however, because multiple exemptions can be applied to a single file. The CSV data file records multiple reasons as a pipe-separated string. First we can look at the most common combinations of reasons.

In [8]:
df2020["reasons"].value_counts()[:25]

33(1)(g)                                      4573
Withheld pending adv                          3398
Parliament Class A                            1285
33(1)(a)                                       497
33(1)(a)|33(1)(b)                              260
Closed period                                  214
33(1)(d)|33(1)(g)                              149
33(1)(a)|33(1)(d)|33(1)(g)                     120
Non Cwlth-no appeal                             54
33(1)(a)|33(1)(b)|Withheld pending adv          53
33(1)(d)                                        50
                                                49
33(1)(a)|Withheld pending adv                   42
33(1)(a)|33(1)(d)|33(1)(e)(ii)|33(1)(g)         27
33(1)(e)(ii)                                    27
33(1)(a)|33(1)(d)|33(1)(e)(i)|33(1)(g)          25
33(1)(e)(ii)|33(1)(g)                           25
33(3)(a)(i)|33(3)(b)|33(3)(a)(ii)|33(3)(b)      24
33(1)(d)|33(1)(e)(iii)|33(1)(g)                 19
33(2)(a)|33(2)(b)              

It's probably more useful, however, to look at the frequency of individual reasons. So we'll split the pip-separated string and create a row for each file/reason combination.

In [9]:
df2020_reasons = df2020.copy()
# Split the reasons field on pipe symbol |. This turns the string into a list of values.
df2020_reasons["reason"] = df2020_reasons["reasons"].str.split("|")

In [10]:
# Now we'll explode the list into separate rows.
df2020_reasons = df2020_reasons.explode("reason")

Now we can look at the frequency of individual reasons. Not, of course, that the sum of the reasons will be greater than the number of files, as some files have multiple exemptions applied to them.

In [11]:
df2020_reasons["reason"].value_counts()

33(1)(g)                5003
Withheld pending adv    3524
Parliament Class A      1286
33(1)(a)                1096
33(1)(d)                 429
33(1)(b)                 362
Closed period            239
33(1)(e)(ii)             110
33(3)(b)                  66
Non Cwlth-no appeal       60
                          49
33(1)(e)(iii)             46
33(3)(a)(i)               38
33(1)(e)(i)               37
33(1)(j)                  30
33(3)(a)(ii)              28
33(2)(a)                  24
33(2)(b)                  24
NRF                       15
MAKE YOUR SELECTION       12
Non Cwlth-depositor       10
Court records              9
33(1)(f)(i)                7
33(1)(f)(ii)               5
33(1)(f)(iii)              4
33(1)(h)                   4
33(1)(c)                   3
Destroyed                  2
Name: reason, dtype: int64

The reasons starting with '33' are clauses in [section 33](http://www8.austlii.edu.au/cgi-bin/viewdoc/au/legis/cth/consol_act/aa198398/s33.html) of the Archives Act. You can look up the Act to find out more about them, or [look at this list](https://www.naa.gov.au/help-your-research/using-collection/access-records-under-archives-act/why-we-refuse-access) on the NAA website. Some of the reasons, such as 'Parliament Class A' refer to particular types of records that are not subject to the same public access arrangements as other government records. Others, such as 'MAKE YOUR SELECTION' seem to be products of the data entry system!

Looking at the other most common reasons:

* 33(1)(g) relates to privacy
* 'Withheld pending adv' is applied to files that are undergoing access examination and have been referred to the relevant government agency for advice on whether they should be released to the public. This is not a final determination – these files may or may not end up being closed. But, as any researcher knows, this process can be *very* slow.
* 33(1)(a) is the national security catch-all

You might also notice that there's a blank line in the list above. This is because some closed files have no reasons recorded in RecordSearch. We can check this.

In [12]:
missing_reasons = df2020.loc[df2020["reasons"] == ""]
missing_reasons.shape[0]

49

There are 46 closed files with no reason recorded. Here's a sample.

In [13]:
missing_reasons.head()

,identifier,series,control_symbol,title,contents_date_str,contents_start_date,contents_end_date,location,access_status,access_decision_date_str,access_decision_date,reasons
612,546435,A1838,919/13/4 PART 31,France - Disarmament - Nuclear Weapons Testing,1972 - 1972,1972-01-01,1972-01-01,Canberra,Closed,07 Oct 2016,2016-10-07,
661,548392,A1838,3127/3/4,South Korea - Labour,1959 - 1968,1959-01-01,1968-01-01,Canberra,Closed,11 Dec 2014,2014-12-11,
862,567499,A1838,563/2/16 PART 9,Radio Australia - Technical - Foreign broadcas...,1961 - 1961,1961-01-01,1961-01-01,Canberra,Closed,29 Aug 2012,2012-08-29,
1351,733600,AWM239,178,[RAN Medical Officers' journals] PENGUIN (1 Ap...,1945 - 1945,1945-01-01,1945-01-01,Australian War Memorial,Closed,14 Apr 2003,2003-04-14,
1560,853102,AWM103,R478/1/147,"[Headquarters, 1st Australian Task Force (HQ 1...",1970 - 1970,1970-01-01,1970-01-01,Australian War Memorial,Closed,22 Jun 2009,2009-06-22,


Let's change the missing reasons to 'None recorded' to make it easier to see what's going on.

In [14]:
df2020_reasons["reason"].replace("", "None recorded", inplace=True)

Let's chart the frequency of the different reasons.

In [15]:
# Once again we'll create a compact dataset for charting
reason_counts = df2020_reasons["reason"].value_counts().to_frame().reset_index()
reason_counts.columns = ["reason", "count"]

In [16]:
# Make the Chart
alt.Chart(reason_counts).mark_bar().encode(
    x="reason", y=alt.Y("count", title="number of files"), tooltip=["reason", "count"]
)

alt.Chart(...)

## Connecting reasons and series

It would be interesting to bring together the analyses above and see how reasons are distributed across series. First we need to reshape our dataset to show combinations of series and reasons.

In [17]:
# Group files by series and reason, then count the number of combinations
series_reasons_counts = (
    df2020_reasons.groupby(by=["series", "reason"]).size().reset_index()
)
# Rename columns
series_reasons_counts.columns = ["series", "reason", "count"]

Now we can chart the results. Once again we'll show the number of files in the 50 most common series, but this time we'll highlight the reasons using color.

In [45]:
alt.Chart(series_reasons_counts).transform_aggregate(
    count="sum(count)",
    groupby=["series", "reason"]
    # Sort by number of files
).transform_window(
    rank="rank(count)",
    sort=[alt.SortField("count", order="descending")]
    # Get the top 50
).transform_filter(
    alt.datum.rank < 50
).mark_bar().encode(
    x=alt.X("series", sort="-y"),
    y=alt.Y("sum(count)", title="number of files", axis=alt.Axis(grid=False)),
    color="reason",
    tooltip=["series", "reason", "count"],
)

alt.Chart(...)

Now we can see that the distribution of reasons varies considerably across series.

## How old are these files?

You would think that the sensitivity of material in closed files diminishes over time. However, there's no automatic re-assessment or time limit on 'closed' files. They stay closed until someone asks for them to be re-examined. That means that some of these files can be quite old. How old? We can use the contents end date to explore this.

In [19]:
# Normalise contents end values as end of year
df2020["contents_end_year"] = df2020["contents_end_date"].apply(
    lambda x: datetime.datetime(x.year, 12, 31)
)

In [20]:
date_counts = df2020["contents_end_year"].value_counts().to_frame().reset_index()
date_counts.columns = ["end_date", "count"]

In [21]:
alt.Chart(date_counts).mark_bar().encode(x="year(end_date):T", y="count").properties(
    width=700
)

alt.Chart(...)

In [22]:
alt.Chart(date_counts.loc[date_counts["end_date"] > "1890-12-31"]).mark_bar().encode(
    x="year(end_date):T", y="count", tooltip="year(end_date)"
).properties(width=700)

alt.Chart(...)

In [23]:
df2020["years_old"] = df2020["contents_end_year"].apply(
    lambda x: round((datetime.datetime.now() - x).days / 365)
)

In [24]:
age_counts = df2020["years_old"].value_counts().to_frame().reset_index()
age_counts.columns = ["age", "count"]
alt.Chart(age_counts.loc[age_counts["age"] < 130]).mark_bar().encode(
    x=alt.X("age:Q", title="age in years"),
    y=alt.Y("count", title="number of files"),
    tooltip=["age", "count"],
).properties(width=700)

alt.Chart(...)

In [25]:
df2020["years_old"].describe()

count    11140.000000
mean        51.081598
std         19.422067
min          6.000000
25%         36.000000
50%         49.000000
75%         65.000000
max        222.000000
Name: years_old, dtype: float64

In [26]:
df2020.loc[df2020["reasons"].str.contains("33(1)(a)", regex=False)][
    "years_old"
].describe()

count    1096.000000
mean       63.380474
std        12.805262
min        22.000000
25%        58.000000
50%        66.000000
75%        73.000000
max        96.000000
Name: years_old, dtype: float64

In [27]:
df2020["years_old"].quantile([0.25, 0.5, 0.75]).to_list()

[36.0, 49.0, 65.0]

## Dates of decisions

In [28]:
df2020["year"] = df2020["access_decision_date"].dt.year
year_counts = df2020["year"].value_counts().to_frame().reset_index()
year_counts.columns = ["year", "count"]

In [29]:
alt.Chart(year_counts).mark_bar().encode(x="year:O", y="count")

alt.Chart(...)

## 33(1)(a)

In [30]:
df331a = df2020.loc[df2020["reasons"].str.contains("33(1)(a)", regex=False)]
df331a.head()

,identifier,series,control_symbol,title,contents_date_str,contents_start_date,contents_end_date,location,access_status,access_decision_date_str,access_decision_date,reasons,contents_end_year,years_old,year
5,140757,A1838,3034/2/2/2 PART 7,Indonesia. Communism in Indonesia,1960 - 1962,1960-01-01,1962-01-01,Canberra,Closed,14 May 2012,2012-05-14,33(1)(a)|33(1)(b)|Withheld pending adv,1962-12-31,60,2012
7,170971,A816,41/301/195,Exchange of staff between Joint Intelligence B...,1953 - 1958,1953-01-01,1958-01-01,Canberra,Closed,29 Apr 1991,1991-04-29,33(1)(a)|33(1)(b),1958-12-31,64,1991
8,171089,A816,43/302/76,Cryptographic Material for ASIO,1951 - 1952,1951-01-01,1952-01-01,Canberra,Closed,11 Mar 1993,1993-03-11,33(1)(a)|33(1)(b),1952-12-31,70,1993
9,171129,A816,44/301/219,SEATO [South East Asia Treaty Organisation] Co...,1957 - 1957,1957-01-01,1957-01-01,Canberra,Closed,01 Aug 1991,1991-08-01,33(1)(a)|33(1)(b),1957-12-31,65,1991
12,200166,A1196,29/501/225,"Evasion of Customs Regulations - RAAF Station,...",1944 - 1944,1944-01-01,1944-01-01,Canberra,Closed,09 Apr 1975,1975-04-09,33(1)(a)|33(1)(b),1944-12-31,78,1975


In [31]:
series_counts_331a = df331a["series"].value_counts().to_frame().reset_index()
series_counts_331a.columns = ["series", "count"]

In [32]:
alt.Chart(series_counts_331a[:50]).mark_bar().encode(
    x=alt.X("series", sort="-y"),
    y=alt.Y("count", title="number of files"),
    tooltip=["series", "count"],
)

alt.Chart(...)

## Withheld pending advice

In [33]:
dfwh = df2020.loc[df2020["reasons"].str.contains("Withheld pending adv", regex=False)]
dfwh.head()

,identifier,series,control_symbol,title,contents_date_str,contents_start_date,contents_end_date,location,access_status,access_decision_date_str,access_decision_date,reasons,contents_end_year,years_old,year
5,140757,A1838,3034/2/2/2 PART 7,Indonesia. Communism in Indonesia,1960 - 1962,1960-01-01,1962-01-01,Canberra,Closed,14 May 2012,2012-05-14,33(1)(a)|33(1)(b)|Withheld pending adv,1962-12-31,60,2012
10,171205,A816,48/301/131,"Inland tele-radio channels in Australia, Papua...",1950 - 1953,1950-01-01,1953-01-01,Canberra,Closed,16 Aug 2018,2018-08-16,Withheld pending adv,1953-12-31,69,2018
11,199284,A1196,2/501/295,Provision of Capacity for the Manufacture of n...,1952 - 1955,1952-01-01,1955-01-01,Canberra,Closed,04 Mar 2020,2020-03-04,Withheld pending adv,1955-12-31,67,2020
13,200647,A1196,36/501/729 PART 2,RAAF Component of the Strategic Reserve- Execu...,1958 - 1958,1958-01-01,1958-01-01,Canberra,Closed,02 Nov 2016,2016-11-02,Withheld pending adv,1958-12-31,64,2016
14,200648,A1196,36/501/729 PART 3,RAAF Component Strategic Reserve. (Execution o...,1958 - 1959,1958-01-01,1959-01-01,Canberra,Closed,02 Nov 2016,2016-11-02,Withheld pending adv,1959-12-31,63,2016


In [34]:
series_counts_wh = dfwh["series"].value_counts().to_frame().reset_index()
series_counts_wh.columns = ["series", "count"]

In [35]:
alt.Chart(series_counts_wh[:50]).mark_bar().encode(
    x=alt.X("series", sort="-y"), y="count"
)

alt.Chart(...)

In [36]:
dfwh["wait_days"] = dfwh["access_decision_date"].apply(
    lambda x: round((datetime.datetime.now() - x).days)
)

/tmp/ipykernel_596751/1245189278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfwh["wait_days"] = dfwh["access_decision_date"].apply(


In [37]:
dfwh["wait_years"] = dfwh["access_decision_date"].apply(
    lambda x: round((datetime.datetime.now() - x).days / 365)
)

/tmp/ipykernel_596751/1295627524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfwh["wait_years"] = dfwh["access_decision_date"].apply(


In [38]:
dfwh["wait_years"].value_counts()

5     652
10    499
7     384
6     376
8     334
2     325
3     275
4     252
11    243
9     114
12     44
13     11
32      2
24      2
19      2
33      1
22      1
30      1
14      1
27      1
31      1
17      1
16      1
15      1
Name: wait_years, dtype: int64

In [39]:
dfwh.loc[dfwh["wait_years"] > 10]

,identifier,series,control_symbol,title,contents_date_str,contents_start_date,contents_end_date,location,access_status,access_decision_date_str,access_decision_date,reasons,contents_end_year,years_old,year,wait_days,wait_years
42,271461,A1838,69/1/3/7 PART 2,USSR Relations with Australia Activities of Au...,1963 - 1964,1963-01-01,1964-01-01,Canberra,Closed,01 Nov 2011,2011-11-01,Withheld pending adv,1964-12-31,58,2011,3913,11
53,273271,A1838,162/11/52 PART 1,Congo. Relations with Other Countries - USSR,1960 - 1960,1960-01-01,1960-01-01,Canberra,Closed,29 Nov 2011,2011-11-29,Withheld pending adv,1960-12-31,62,2011,3885,11
54,273272,A1838,162/11/73 PART 2,Congo. Relations with Other Countries. Congo -...,1960 - 1961,1960-01-01,1961-01-01,Canberra,Closed,29 Nov 2011,2011-11-29,Withheld pending adv,1961-12-31,61,2011,3885,11
60,277191,A6980,S250793,Non-British European Migration from China Part 6,1968 - 1979,1968-01-01,1979-01-01,Canberra,Closed,04 Aug 2010,2010-08-04,Withheld pending adv,1979-12-31,43,2010,4367,12
74,302544,A1209,1957/4254,"ANZUS Council meeting - Washington, November 1956",1956 - 1956,1956-01-01,1956-01-01,Canberra,Closed,06 Apr 2010,2010-04-06,33(1)(a)|33(1)(b)|Withheld pending adv,1956-12-31,66,2010,4487,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8941,30091158,A12381,20/47/1,RCIS - Prime Minister Mr Fraser 1976,1975 - 1976,1975-01-01,1976-01-01,Canberra,Closed,13 Apr 2010,2010-04-13,Withheld pending adv,1976-12-31,46,2010,4480,12
9059,30713375,A9737,1991/766 PART 1,French Polynesia - nuclear testing,25 Jul 1978 - 29 May 1991,1978-07-25,1991-05-29,Canberra,Closed,16 Jan 2012,2012-01-16,Withheld pending adv,1991-12-31,31,2012,3837,11
9073,30714039,A9737,1990/1314 PART 1,French nuclear testing,27 Feb 1973 - 15 Mar 1995,1973-02-27,1995-03-15,Canberra,Closed,15 Dec 2011,2011-12-15,Withheld pending adv,1995-12-31,27,2011,3869,11
9340,31162900,A4626,26,Department - Attorney-General's,18 Jul 1975 - 14 Apr 1976,1975-07-18,1976-04-14,Canberra,Closed,29 Jun 2011,2011-06-29,Withheld pending adv,1976-12-31,46,2011,4038,11


In [40]:
dfwh["wait_years"].describe()

count    3524.000000
mean        6.547673
std         3.039012
min         2.000000
25%         5.000000
50%         6.000000
75%         9.000000
max        33.000000
Name: wait_years, dtype: float64

In [41]:
dfwhs = df2020.loc[df2020["reasons"] == "Withheld pending adv"]

In [42]:
dfwhs["wait_years"] = dfwhs["access_decision_date"].apply(
    lambda x: round((datetime.datetime.now() - x).days / 365)
)

/tmp/ipykernel_596751/1357762301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfwhs["wait_years"] = dfwhs["access_decision_date"].apply(


In [43]:
dfwhs["wait_years"].describe()

count    3398.000000
mean        6.429665
std         2.778587
min         2.000000
25%         5.000000
50%         6.000000
75%         8.000000
max        24.000000
Name: wait_years, dtype: float64

In [44]:
dfwhs.loc[dfwhs["wait_years"] > 10]

,identifier,series,control_symbol,title,contents_date_str,contents_start_date,contents_end_date,location,access_status,access_decision_date_str,access_decision_date,reasons,contents_end_year,years_old,year,wait_years
42,271461,A1838,69/1/3/7 PART 2,USSR Relations with Australia Activities of Au...,1963 - 1964,1963-01-01,1964-01-01,Canberra,Closed,01 Nov 2011,2011-11-01,Withheld pending adv,1964-12-31,58,2011,11
53,273271,A1838,162/11/52 PART 1,Congo. Relations with Other Countries - USSR,1960 - 1960,1960-01-01,1960-01-01,Canberra,Closed,29 Nov 2011,2011-11-29,Withheld pending adv,1960-12-31,62,2011,11
54,273272,A1838,162/11/73 PART 2,Congo. Relations with Other Countries. Congo -...,1960 - 1961,1960-01-01,1961-01-01,Canberra,Closed,29 Nov 2011,2011-11-29,Withheld pending adv,1961-12-31,61,2011,11
60,277191,A6980,S250793,Non-British European Migration from China Part 6,1968 - 1979,1968-01-01,1979-01-01,Canberra,Closed,04 Aug 2010,2010-08-04,Withheld pending adv,1979-12-31,43,2010,12
622,546984,A1838,938/17/7/1 PART 2,United Nations - Law of the Sea - Pollution of...,1971 - 1972,1971-01-01,1972-01-01,Canberra,Closed,21 Sep 2011,2011-09-21,Withheld pending adv,1972-12-31,50,2011,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8941,30091158,A12381,20/47/1,RCIS - Prime Minister Mr Fraser 1976,1975 - 1976,1975-01-01,1976-01-01,Canberra,Closed,13 Apr 2010,2010-04-13,Withheld pending adv,1976-12-31,46,2010,12
9059,30713375,A9737,1991/766 PART 1,French Polynesia - nuclear testing,25 Jul 1978 - 29 May 1991,1978-07-25,1991-05-29,Canberra,Closed,16 Jan 2012,2012-01-16,Withheld pending adv,1991-12-31,31,2012,11
9073,30714039,A9737,1990/1314 PART 1,French nuclear testing,27 Feb 1973 - 15 Mar 1995,1973-02-27,1995-03-15,Canberra,Closed,15 Dec 2011,2011-12-15,Withheld pending adv,1995-12-31,27,2011,11
9340,31162900,A4626,26,Department - Attorney-General's,18 Jul 1975 - 14 Apr 1976,1975-07-18,1976-04-14,Canberra,Closed,29 Jun 2011,2011-06-29,Withheld pending adv,1976-12-31,46,2011,11
